## White listed files

This notebook demonstrates that some files in the DDP dataset may be suitable for analysis. 

#### Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas   # data analysis and manipulation
import numpy    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
import glob
import numpy as np
import pandas as pd
from numpy.fft import fft, fftfreq
from scipy import signal

import matplotlib
import matplotlib.pyplot as plt

from mne.time_frequency.tfr import morlet
from mne.viz import plot_filter, plot_ideal_filter


from IPython.display import clear_output
import warnings

import sys

Below eegyolk is importted in it's most updated version
But the cell could be replaced by importing the stable library and then importing the modules of it.

In [2]:

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
#import initialization_functions #library to import data
import epod_helper
import rawf
from config import Config
from rawf import RawData

#### Load EEG files
Change your config file to change where data is coming from

In [3]:
config = Config()


In [4]:
# data_file_directory = config.get_directory('data')
# data_file_directory

In [5]:
# files = glob.glob('/volume-ceph/DDP_projectfolder/*.cnt', 
#                    recursive = True)
# print(files)

In [6]:
# print('\nNamed with wildcard *:')
# for name in glob.glob(os.path.join(data_file_directory, '/*'):
#     print(name)

In [7]:
#glob.glob(data_file_directory)

In [13]:
#eeg_file_root = "../../volume-ceph/ePodium_projectfolder"
acquired = RawData(config.get_directory('data'), config.get_directory('metadata'))

Note we assume you are looking at cnt files

In [14]:
acquired.raw.head()

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
0,35,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,035_11_jc_mmn36_slp_mmn25_slp,11,331.0,11.033333,0.919444
1,27,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,027_11_jc_mmn25_wk,11,326.0,10.866667,0.905556
2,25,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,025_11_mc_mmn36_wk,11,360.0,12.000000,1.000000
3,35,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,035_11_jc_mmn36slp_mmn25_slp_2,11,331.0,11.033333,0.919444
4,30,/volume-ceph/DDP_projectfolder/dataset/11mnd m...,030_11_jc_mmn36_wk_mmn25_wk,11,328.0,10.933333,0.911111


Now we used the default settings above, let's just switch the settings and compare

In [10]:
def read_raw_agnostic(fname, int_arg):
    """
    For testing purposes
    """
    
    cnt_read_args = {
        'eog': 'auto',
        'data_format': int_arg,
        'date_format': 'dd/mm/yy',
        'verbose': False,
    }
    read = mne.io.read_raw_cnt(
            fname,
            data_format=int_arg,
            preload=True,
            
        )
    return read

In [22]:
# for file in acquired.raw.cnt_path:
#      print(mne.utils.fill_doc(file))

In [24]:
for file in acquired.raw.cnt_path:
     love = mne.io._read_annotations_cnt(file, data_format='int16')

AttributeError: module 'mne.io' has no attribute '_read_annotations_cnt'

In [16]:
read_on_32 = read_raw_agnostic(paths_df.cnt_path[n], 'int32')


NameError: name 'paths_df' is not defined

In [ ]:
raw_df32 = read_on_32.to_data_frame()
raw_df32.head()

In [ ]:
read_on_16 = read_raw_agnostic(paths_df.cnt_path[n], 'int16')
raw_df16 = read_on_16.to_data_frame()
raw_df16.head()

In [ ]:
# The big difference

In [ ]:
(raw_df32 -raw_df16).sum().sum()

### Youch!

So here we saw an example of how reading the same file on int32 and int16 will give us dramatically different results. We must know which we have when we use cnt files.

## This is no floating point error!

So is there anything we can do to figure out which kind of files we had?
Why, yes of course. First of all on a few files in the DDP dataset there is information on this
in the metadata (numchannels). Second of all, we can be a bit sneaky, and ask ourselves if
32 is a possibility. When we look at the number of bit and bytes inside the file for the
information, is this a number that we could divide by 32? Well it turns out when you peel the header off of
the files you can calculate the number of bits/bytes inside. And they are (all tested so far) with a bit
number divisible by 32 (or a byte number divisible by 4). But wait, that means de-facto , they
are also divisble by 16. So we can say probably the files should have been read as 32; but
we can not say with 100% certainty. This is not a problem outside the cnt format, but for cnt formatted
data, back to cell 1.

In [ ]:
4*8